In [1]:
import sys
sys.path.insert(0, os.path.abspath('../'))
import os
import pyspark as spark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
from pyspark.sql import DataFrame, Row
from functools import reduce
import functools
from pyspark.sql.types import StructType, StructField, StringType, LongType, ArrayType, IntegerType, MapType
from pyspark.sql.functions import *

In [16]:
from datetime import datetime

def get_current_datetime_map():
    now = datetime.now()
    now.strftime("%Y-%m-%d_%H:%M:%S")
    date_time = {}
    date_time['year'] = now.strftime("%Y")
    date_time['month'] = now.strftime("%m")
    date_time['day'] = now.strftime("%d")
    date_time['time'] = now.strftime("%H:%M:%S")
    return date_time

print(get_current_datetime_map())

{'year': '2023', 'month': '01', 'day': '05', 'time': '18:13:55'}


In [17]:
events = {'domain':'pets'}
if 'domain' in events:
    print(events['domain'])

pets


In [2]:
from graphframes import *
from difflib import get_close_matches

In [3]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
conf=SparkConf()
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

22/12/29 15:04:56 WARN Utils: Your hostname, preacher resolves to a loopback address: 127.0.1.1; using 192.168.0.101 instead (on interface enp4s0)
22/12/29 15:04:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/29 15:04:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/preacher/anaconda3/envs/deltarun/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
def get_data_from_s3(s3_bucket, file_key):
    df = spark.read.format('csv').option('header','true').load(s3_bucket+file_key)
    print(df.count())
    return df

def get_data_from_athena(database, table_name):
    df = spark.read.format('csv').option('header','true').load(database+table_name)
    print(df.count())
    return df

def delete_s3_data( s3_bucket, vertex_path):
    return

def saving_df_to_s3(df, s3_bucket, file_key):
    df.write.mode("overwrite").option("header",True).csv(s3_bucket+file_key)
    return

def get_data_from_aurora_db(s3_bucket, cluster_s3_path):
    df = spark.read.format('csv').option('header','true').load(cluster_s3_path)
    print(df.count())
    return df

In [37]:
def convert_to_database_schema(cluster_df: DataFrame, brand_dict: DataFrame, db_brand_list:DataFrame, cluster_type: str)-> DataFrame:
    if ( cluster_df == None or len(cluster_df.head(1)) == 0 ) or ( brand_dict == None or len(brand_dict.head(1)) == 0 ):
        return None
        
    cluster_brand_info = cluster_df.join(brand_dict, cluster_df.brand == brand_dict.crawled_name, 'inner').drop('brand')
    cluster_brand_info = cluster_brand_info.withColumn('source', lit(cluster_type))
    cluster_brand_info = cluster_brand_info.withColumnRenamed('id', 'cluster_id')
    cluster_brand_info = cluster_brand_info.withColumn('status', lit('IN_PROGRESS'))
    cluster_brand_info = cluster_brand_info.withColumn('updated_date', current_date().cast("string"))
    cluster_brand_info = cluster_brand_info.select('cluster_id', 'crawled_name', 'source', 'status', 'updated_date', 'product_url_1', 'image_url_1', 'product_url_2')
    updation_brands = cluster_brand_info.filter(cluster_brand_info.crawled_name.isin(db_brand_list))
    insertion_brands = cluster_brand_info.filter(~cluster_brand_info.crawled_name.isin(db_brand_list))
    updation_brnads = updation_brands.withColumn( 'query_type', lit('U'))
    insertion_brnads = insertion_brands.withColumn('query_type', lit('I'))
    return insertion_brnads, updation_brnads

In [6]:
def combine_dfs(df_list):
    print("Combning dfs")
    combined_df = reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), df_list)
    return combined_df

def get_max_id(df):
    if df == None or len(df.head(1)) == 0:
        return 0

    result = df.select([max('id')])
    max_id = result.collect()[0]['max(id)']
    return int(max_id)

def insert_id(df, start_idx):
    print("Getting ids for new uuids")
    updated_schema = StructType(df.schema.fields[:] + [StructField("id", LongType(), False)])
    zipped_rdd = df.rdd.zipWithIndex().map(lambda x: (x[0], x[1] + start_idx))
    id_df = (zipped_rdd.map(lambda ri: Row(*list(ri[0]) + [ri[1]])).toDF(updated_schema))
    # print("New id_uuid = {}".format(new_id_uuid_df.count()))
    return id_df

In [7]:
def create_sku_uuid(uuid):
    arr = uuid.split("<>")
    arr = arr[1:len(arr)]
    return "<>".join(arr)
    
def create_sku_uuid_column(df):
    sku_uuid = udf(create_sku_uuid, StringType())
    res_df = df.withColumn("sku_uuid", sku_uuid(df.uuid))
    return res_df

def ml_input_preprocessing_preprocessing(ml_input):
    required_store = ['petcarerx', 'petsmart', 'lovegroomers', 'ryanspet', 'tractorsupplyrx', 'californiapetpharmacy', 'petsuppliesplus', 'petedge', 'petmeds', 'entirelypetspharmacy', 'chewy', 'vcahospitals', 
    'heartlandvetsupply', 'petco', 'allivet', 'smartpakequine', 'petflow', 'jefferspet', 'tractorsupply', 'entirelypets', 'groomerschoice', 'baxterboo', 'chewy']
    ml_input = ml_input.filter(ml_input.store_name.isin(required_store))
    ml_input = create_sku_uuid_column(ml_input)
    ml_input = ml_input.dropDuplicates(['sku_uuid'])
    print(ml_input.count())
    ml_input = ml_input.na.drop(subset=["brand"])
    print(ml_input.count())
    return ml_input

In [8]:
def generate_brand_dictionary(ml_input, store_priority_order):
    print("Generating brand dictionary")
    ml_input = ml_input.drop('sku','uuid','sku_uuid')
    ml_input = ml_input.dropDuplicates(['brand','store_name'])
    ml_input = ml_input.join(store_priority_order, ml_input.store_name == store_priority_order.store, 'inner').drop('store')
    brand_dict = ml_input.groupBy('brand').agg( collect_set(ml_input.store_name).alias('store_list'), sort_array(collect_set(struct( ml_input.priority, ml_input.product_url, ml_input.image_url))).alias('brand_info'))
    brand_dict = brand_dict.withColumn('brand_info', when(size(brand_dict.brand_info) == 0, array()).when(size(brand_dict.brand_info) == 1,  array(brand_dict.brand_info[0])).otherwise(array(brand_dict.brand_info[0], brand_dict.brand_info[1])))
    brand_dict = brand_dict.select( brand_dict.brand.alias('crawled_name'), \
            brand_dict.store_list,\
            brand_dict.brand_info[0].product_url.alias('product_url_1'), \
            brand_dict.brand_info[0].image_url.alias('image_url_1'),\
            brand_dict.brand_info[1].product_url.alias('product_url_2'))

    print("brand dictionary generated")
    return brand_dict
    # brand_dict['crawled_name','source_store_list', 'product_url_1', 'product_url_2', 'image_url']

In [9]:
def product_matches_preprocessing(product_matches, ml_input):
    product_matches = product_matches.select('sku_uuid_a', 'sku_uuid_b')
    ml_input = ml_input.withColumnRenamed('sku_uuid','ml_input_sku_uuid')
    sku_uuid_a_matched_data = product_matches.join(ml_input, ml_input.ml_input_sku_uuid == product_matches.sku_uuid_a, 'inner').select('sku_uuid_a','sku_uuid_b',col('brand').alias('brand_a'))
    sku_uuid_a_and_b_matched_data = sku_uuid_a_matched_data.join(ml_input, ml_input.ml_input_sku_uuid == sku_uuid_a_matched_data.sku_uuid_b, 'inner').select('sku_uuid_a', 'sku_uuid_b', 'brand_a', col('brand').alias('brand_b'))
    return sku_uuid_a_and_b_matched_data

In [10]:
def generate_brand_matches_from_product_matches(product_matches):
    directed_a_b_brand_matches = product_matches
    directed_b_a_brand_matches = directed_a_b_brand_matches.select(col('sku_uuid_a').alias('sku_uuid_b'), col('sku_uuid_b').alias('sku_uuid_a'),  col('brand_a').alias('brand_b'), col('brand_b').alias('brand_a'))
    undirected_brand_matches = combine_dfs([directed_a_b_brand_matches, directed_b_a_brand_matches]).dropDuplicates(['sku_uuid_a','sku_uuid_b'])
    undirected_brand_matches = undirected_brand_matches.groupBy(['brand_a','brand_b']).count()
    return undirected_brand_matches

In [11]:
def generate_graph_edges(undirected_brand_matches):
    edge_df = undirected_brand_matches.withColumnRenamed('brand_a','src').withColumnRenamed('brand_b','dst')
    return edge_df

In [12]:
def generate_graph_vertices(brand_dict):
    vertex_df = brand_dict.select(brand_dict.crawled_name.alias('id'))
    return vertex_df

In [13]:
class Graph:
    def __init__(self, vertices, edges):
        self.vertices = vertices
        self.edges = edges
        self.construct_graph()
    
    def construct_graph(self):
        sc.setCheckpointDir("/home/preacher/Bungee/bnt/data_set/checkpoints")
        self.graph = GraphFrame(self.vertices, self.edges)
        return self.graph
        
    def get_connected_componenets(self):
        sc.setCheckpointDir("/home/preacher/Bungee/bnt/data_set/checkpoints")
        self.connected_components = self.graph.connectedComponents()
        return self.connected_components
        #  connected_components["id","componenets"]

    def generate_filtered_clusters(self):
        self.generate_clusters()
        self.filtered_cluster = self.connected_component_cluster.filter(size(col("cluster"))>1)
        self.filtered_cluster = self.filtered_cluster.filter(size(col("cluster"))<50) 
        return self.filtered_cluster


    def generate_clusters(self):
        self.get_connected_componenets()
        self.connected_component_cluster = self.connected_components.groupBy('component').agg(collect_set('id').alias("cluster")).drop('component')
        #  one sinle brand cannot form a clusters so removing single size array
        return self.connected_component_cluster

In [14]:
class Brand_normalization:
    def __init__(self, new_brand_cluster, old_brand_cluster ):
        self.new_cluster_start_index = 1
        self.old_brand_cluster = self.get_old_brand_sets(old_brand_cluster)
        self.new_brand_cluster = self.get_new_brand_sets(new_brand_cluster)

    def generate_brand_clusters(self):
        if self.old_brand_cluster != None:
            self.new_brand_cluster, self.updated_brand_cluster = self.get_updated_brand_sets()
            return self.new_brand_cluster, self.updated_brand_cluster
        else :
            self.new_brand_cluster = self.new_brand_cluster.withColumnRenamed('new_id','id').withColumnRenamed('new_brand','brand')
            self.updated_brand_cluster = None
            return self.updated_brand_cluster, None

    def get_old_brand_sets(self, old_brand_cluster):
        if old_brand_cluster == None:
            return None
        self.new_cluster_start_idex = get_max_id(old_brand_cluster) + 1
        return old_brand_cluster.select(old_brand_cluster.id.alias('old_id'), old_brand_cluster.brand.alias('old_brand'))

    def get_new_brand_sets(self, new_brand_cluster):
        new_brand_cluster =  insert_id(new_brand_cluster, self.new_cluster_start_idex)
        return new_brand_cluster.select(new_brand_cluster.id.alias('new_id'), explode(new_brand_cluster.cluster).alias('new_brand'))

    def get_vertex(self, set_id_matches):
        set_id_a = set_id_matches.select(col("new_id").alias("id"))
        set_id_b = set_id_matches.select(col("old_id").alias("id"))
        vertex = combine_dfs([set_id_a, set_id_b]).dropDuplicates(['id'])
        return vertex
    
    def generate_graph_edges(self, set_id_matches):
        directed_edge_a_b = set_id_matches.select('new_id','old_id')
        directed_edge_b_a = set_id_matches.select(col('new_id').alias('old_id') ,col('old_id').alias('new_id'))
        undirected_edges = combine_dfs([directed_edge_a_b, directed_edge_b_a])
        undirected_edges = undirected_edges.dropDuplicates(["new_id","old_id"]).withColumnRenamed('new_id', 'src').withColumnRenamed('old_id', 'dst')
        return undirected_edges

    def get_brand_cluster(self, connected_clusters):
        old_cluster_brands = connected_clusters.join(self.old_brand_cluster, connected_clusters.id == self.old_brand_cluster.old_id, 'inner').drop('old_id')
        old_cluster_brands = old_cluster_brands.withColumnRenamed('old_brand','brand')
        new_cluster_brands = connected_clusters.join(self.new_brand_cluster, connected_clusters.id == self.new_brand_cluster.new_id, 'inner').drop('new_id')
        new_cluster_brands = new_cluster_brands.withColumnRenamed('new_brand','brand')
        connected_brand_clusters = combine_dfs([old_cluster_brands, new_cluster_brands])
        return connected_brand_clusters
        #  connected_brand_clusters("id", "")

    def get_updated_brand_sets(self):
        set_id_matches = self.new_brand_cluster.join(self.old_brand_cluster, self.new_brand_cluster.new_brand == self.old_brand_cluster.old_brand, 'inner')
        vertices = self.get_vertex(set_id_matches)
        edges = self.generate_graph_edges(set_id_matches)
        graph = Graph(vertices, edges)
        connected_clusters = graph.get_connected_componenets()

        connected_brand_clusters = self.get_brand_cluster(connected_clusters)
        set_id_list_brand_cluster = connected_brand_clusters.groupBy('component').agg(array_sort(collect_set('id')).alias('id_sets'), collect_set('brand').alias('brand_set')).drop('component')
        self.updated_id_brand_cluster = set_id_list_brand_cluster.withColumn('id',array_min(set_id_list_brand_cluster.id_sets)).drop('id_sets')

        # updated_id_brand_cluster['id','brand_set']
        old_brand_set = self.old_brand_cluster.groupBy('old_id').agg(collect_set('old_brand').alias('old_brand_set'))
        set_id_old_and_updated_brand_cluster = self.updated_id_brand_cluster.join(old_brand_set, self.updated_id_brand_cluster.id == old_brand_set.old_id, 'inner' ).drop('old_id')
        updated_brand_cluster = set_id_old_and_updated_brand_cluster.select("id",array_except("brand_set","old_brand_set").alias('brand_set')).filter(size(col("brand_set"))>0)
        updated_brand_cluster = updated_brand_cluster.withColumn('brand', explode('brand_set')).drop('brand_set')

        # updation_brand_cluster['id','brand_set']
        old_and_updated_set_id = connected_brand_clusters.select(col('id').alias('old_set_id')).dropDuplicates()
        new_set_id_brand_cluster = self.new_brand_cluster.groupBy(col("new_id").alias('id')).agg(collect_set('new_brand').alias('brand_set'))
        new_brand_cluster = new_set_id_brand_cluster.join(old_and_updated_set_id, new_set_id_brand_cluster.id == old_and_updated_set_id.old_set_id, 'leftanti')
        new_brand_cluster = new_brand_cluster.withColumn('brand', explode('brand_set')).drop('brand_set')
        # insertion_brand_cluster['id','brand_set']
        return new_brand_cluster, updated_brand_cluster
    
    def get_updated_brand_cluster(self):
        if self.new_brand_cluster is None or len(self.new_brand_cluster.head(1)) == 0:
            self.merged_brand_cluster = self.updated_brand_cluster
        elif self.updated_brand_cluster is None or len(self.updated_brand_cluster.head(1)) == 0:
            self.merged_brand_cluster = self.new_brand_cluster
        else:
            old_cluster = self.old_brand_cluster.join(self.updated_brand_cluster, self.old_brand_cluster.old_brand == self.updated_brand_cluster.brand, 'leftanti')
            old_cluster = old_cluster.withColumnRenamed('old_id', 'id').withColumnRenamed('old_brand', 'brand')
            self.merged_brand_cluster = combine_dfs([old_cluster, self.updated_brand_cluster, self.new_brand_cluster])
        
        return self.merged_brand_cluster
        # latest_brand_cluster['id','brand']


In [15]:
class String_Clustering:
    def  __init__(self, brand_cluster, brand_dict):
        self.normalized_brands = brand_cluster # latest_brand_cluster['id','brand_id_set']
        self.brand_dict = brand_dict
        self.crawled_brands = self.get_crawled_brands_list()

    def get_crawled_brands_list(self):
        return self.brand_dict.select( self.brand_dict.crawled_name.alias('brand'))
        # crawled_brands['id','brand']

    def get_unnormalized_brands(self):
        return self.crawled_brands.join(self.normalized_brands, self.crawled_brands.brand == self.normalized_brands.brand, 'leftanti')
        # crawled_brands['id','brand']

    def get_string_clusters(self, brand_list):
        clusters = []
        for brand in brand_list:
            cluster = get_close_matches(brand, brand_list)
            clusters.append(cluster)

        cluster_string = [ [','.join(cluster)] for cluster in clusters]
        column = ['string_clusters']
        brand_string_df = spark.createDataFrame(cluster_string, column)
        brand_string_cluster_df = brand_string_df.select(array_sort(split(col("string_clusters"),",")).alias("brand_cluster")).drop("string_clusters")
        brand_string_cluster_df = brand_string_cluster_df.filter(size(brand_string_cluster_df.brand_cluster)>1).drop_duplicates()
        brand_string_cluster_df.printSchema()
        return brand_string_cluster_df

    def cluster_unnormalized_brands(self):
        self.unclustered_brands = self.get_unnormalized_brands()
        unclustered_brands_list = self.unclustered_brands.rdd.map(lambda x:x[0]).collect()
        string_cluster = self.get_string_clusters(unclustered_brands_list)
        idx = get_max_id(self.normalized_brands)
        id_string_cluster = insert_id(string_cluster, idx+1)
        self.id_string_cluster = id_string_cluster.withColumn('brand', explode('brand_cluster')).drop('brand_cluster')
        return self.id_string_cluster

    def get_unclustered_brands(self):
        if self.id_string_cluster == None:
            return self.unclustered_brands
        else :
            self.unclustered_brands = self.unclustered_brands.subtract(self.id_string_cluster.select('brand'))
            return self.unclustered_brands 

In [90]:
bungeedatalake_path = "/home/preacher/Bungee/bnt/data_set/saas_team/crawled_data.csv"
exact_matches_path = "/home/preacher/Bungee/bnt/data_set/saas_team/exact_matches.csv"

<h2>Input</h2>

In [16]:
s3_bucket = '/home/preacher/Bungee/bnt/data_set/'
run_type = 'NEW'
exact_matches_database = '/home/preacher/Bungee/CodeRepo/kaushik_forked_repo/kaushik_dev/bnt/data_set/'
exact_matches_table_name = 'exact_matches_run_2.csv'
bungeedatalake_database = '/home/preacher/Bungee/CodeRepo/kaushik_forked_repo/kaushik_dev/bnt/data_set/'
bungeedatalake_table_name = 'datalake_run_2.csv'

<h2>Data Fetching</h2>

In [17]:
cluster_s3_path = ''
vertex_path = ''
exact_matches = get_data_from_athena(exact_matches_database, exact_matches_table_name)
bungeedatalake = get_data_from_athena(bungeedatalake_database, bungeedatalake_table_name)


8
24


In [18]:
exact_matches.show(truncate=False, n = 16)
bungeedatalake.show(truncate=False, n = 16)

+--------------------------------------------------+-----------------------------------------------------+------------------------------------+---------------------------------------+-----+-----------------+-----------------+
|uuid_a                                            |uuid_b                                               |sku_uuid_a                          |sku_uuid_b                             |score|base_source_store|comp_source_store|
+--------------------------------------------------+-----------------------------------------------------+------------------------------------+---------------------------------------+-----+-----------------+-----------------+
|sku_pb002554<>pb002554<>entirelypets<>entirelypets|sku_328012055<>328012055<>walmart<>walmart           |pb002554<>entirelypets<>entirelypets|328012055<>walmart<>walmart            |0.9  |walmart          |walmart          |
|sku_917115167<>917115167<>walmart<>walmart        |000005274508<>146940<>chewy<>chewy          

In [94]:
abc = exact_matches.show(16, truncate=False)
#  write.mode("overwrite").option("header",True).parquet('/home/preacher/Bungee/bnt/data_set/saas_team/brand_dict') -->

+--------------------------------------------------+-----------------------------------------------------+------------------------------------+---------------------------------------+-----+-----------------+-----------------+
|uuid_a                                            |uuid_b                                               |sku_uuid_a                          |sku_uuid_b                             |score|base_source_store|comp_source_store|
+--------------------------------------------------+-----------------------------------------------------+------------------------------------+---------------------------------------+-----+-----------------+-----------------+
|sku_pb002554<>pb002554<>entirelypets<>entirelypets|sku_328012055<>328012055<>walmart<>walmart           |pb002554<>entirelypets<>entirelypets|328012055<>walmart<>walmart            |0.9  |walmart          |walmart          |
|sku_917115167<>917115167<>walmart<>walmart        |000005274508<>146940<>chewy<>chewy          

In [95]:
abc

<h2>Data Preprocessing</h2>

In [19]:

filtered_crawled_data = ml_input_preprocessing_preprocessing(bungeedatalake)
filtered_product_matches = product_matches_preprocessing(exact_matches, filtered_crawled_data)

24
24


In [97]:
filtered_crawled_data.count()
filtered_product_matches.count()


8

<h2>Vertex and Edge Generation</h2>

In [98]:
filtered_crawled_data.show(n=20, truncate=False)
filtered_product_matches.show(n=20, truncate=False)

+-----------------------------------------------------+-------------+-------------------+----------+-----------------------------------------------------------------------------------------------------------+---------------------------------------------------+---------------------------------------+
|uuid                                                 |store_name   |brand              |sku       |image_url                                                                                                  |product_url                                        |sku_uuid                               |
+-----------------------------------------------------+-------------+-------------------+----------+-----------------------------------------------------------------------------------------------------------+---------------------------------------------------+---------------------------------------+
|000005274508<>146940<>chewy<>chewy                   |tractorsupply|apple corp ltd     |146940  

In [20]:
def get_store_priority_order():
    store_priority_order = [['petcarerx', 0], ['petsmart',1], ['lovegroomers',2], ['ryanspet',3], ['tractorsupplyrx',4], ['californiapetpharmacy',5], ['petsuppliesplus',6], ['petedge',7], ['petmeds',8], ['entirelypetspharmacy',9], ['chewy',10], ['vcahospitals',11], 
        ['heartlandvetsupply',12], ['petco',13], ['allivet',14], ['smartpakequine',15], ['petflow',16], ['jefferspet',17], ['tractorsupply',18], ['entirelypets',19], ['groomerschoice',20], ['baxterboo',21], ['chewy',22]]
    column = ['store','priority']
    store_priority_order = spark.createDataFrame(store_priority_order, column)
    return store_priority_order

In [21]:
store_priority_order = get_store_priority_order()
brand_dict = generate_brand_dictionary(filtered_crawled_data, store_priority_order)
brand_matches = generate_brand_matches_from_product_matches(filtered_product_matches)

Generating brand dictionary
brand dictionary generated
Combning dfs


In [101]:
brand_dict.printSchema()

root
 |-- crawled_name: string (nullable = true)
 |-- store_list: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- product_url_1: string (nullable = true)
 |-- image_url_1: string (nullable = true)
 |-- product_url_2: string (nullable = true)



In [22]:
vertice_df = generate_graph_vertices(brand_dict)
edge_df = generate_graph_edges(brand_matches)

In [103]:
vertice_df.printSchema()
vertice_df.show()
edge_df.printSchema()
edge_df.show()


root
 |-- id: string (nullable = true)

+--------------------+
|                  id|
+--------------------+
|            dry shod|
|           Goodwinol|
|             handson|
|Pure Sole Hoof Pr...|
|               touch|
|     perfect prep eq|
|               apple|
|             trudell|
|           apple inc|
|            pro plan|
|              purina|
|          pro purina|
|        dryshod west|
|                purr|
| handson equine. Llc|
|     Perfect-product|
|             aerokat|
|            aerodawg|
|      apple corp ltd|
|      handson equine|
+--------------------+
only showing top 20 rows

root
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- count: long (nullable = false)

+-------------------+-------------------+-----+
|                src|                dst|count|
+-------------------+-------------------+-----+
|           aerodawg|            trudell|    1|
|     handson equine|            handson|    1|
|            handson|     hand

In [104]:
vertice_df.printSchema()

root
 |-- id: string (nullable = true)



In [105]:
edge_df.printSchema()

root
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- count: long (nullable = false)



<h2>Graph Generation</h2>


In [23]:
brand_graph = Graph(vertice_df, edge_df)
brand_cluster = brand_graph.generate_filtered_clusters()
brand_cluster.show()
# brand_cluster['id_set']


+--------------------+
|             cluster|
+--------------------+
|[trudell medicin,...|
|[perfect prep eq,...|
|[handson, handson...|
|[apple corp ltd, ...|
|  [averapack, touch]|
| [aerodawg, trudell]|
|  [purr, pro purina]|
+--------------------+



In [24]:
def generate_existing_graph(aurora_db_data):
    if aurora_db_data == None:
        return None
    old_cluster = aurora_db_data.select(aurora_db_data.crawled_name.alias('brand'), aurora_db_data.cluster_id.cast(IntegerType()).alias('id'))
    return old_cluster

In [25]:
def get_brnads_in_db(aurora_db_df):
    if aurora_db_df == None:
        return [] 
    
    db_brands = aurora_db_df.rdd.map(lambda df: df.crawled_name).collect()
    return db_brands

In [26]:
cluster_s3_path = '/home/preacher/Bungee/CodeRepo/kaushik_forked_repo/kaushik_dev/bnt/data_set/insertion_brand_match_cluster/part-00000-4222cd0d-b04a-480d-af9e-32b64ff331e8-c000.csv'

In [27]:
aurora_db_data = get_data_from_aurora_db(s3_bucket, cluster_s3_path) #if run_type == 'DAILY' else None
old_brand_cluster = generate_existing_graph(aurora_db_data)
db_brand_list = get_brnads_in_db(aurora_db_data)
old_brand_cluster
# old_brand_cluster['old_id','old_brand']//exploded brand_id cluster

15


DataFrame[brand: string, id: int]

In [28]:
db_brand_list

['dry shod',
 'handson',
 'perfect prep eq',
 'apple',
 'apple inc',
 'pro plan',
 'purina',
 'pro purina',
 'dryshod west',
 'purr',
 'handson equine. Llc',
 'Perfect-product',
 'aerokat',
 'aerodawg',
 'handson equine']

In [137]:
old_brand_cluster.show()
old_brand_cluster.printSchema()

+-------------------+---+
|              brand| id|
+-------------------+---+
|           dry shod|  7|
|            handson|  6|
|    perfect prep eq|  2|
|              apple|  3|
|          apple inc|  3|
|           pro plan|  4|
|             purina|  5|
|         pro purina|  4|
|       dryshod west|  7|
|               purr|  5|
|handson equine. Llc|  6|
|    Perfect-product|  2|
|            aerokat|  1|
|           aerodawg|  1|
|     handson equine|  6|
+-------------------+---+

root
 |-- brand: string (nullable = true)
 |-- id: integer (nullable = true)



In [138]:
old_brand_cluster.groupBy('id').agg(collect_set('brand')).show(truncate=False)

+---+----------------------------------------------+
|id |collect_set(brand)                            |
+---+----------------------------------------------+
|1  |[aerodawg, aerokat]                           |
|6  |[handson equine. Llc, handson, handson equine]|
|3  |[apple, apple inc]                            |
|5  |[purina, purr]                                |
|4  |[pro plan, pro purina]                        |
|7  |[dry shod, dryshod west]                      |
|2  |[perfect prep eq, Perfect-product]            |
+---+----------------------------------------------+



In [139]:
brand_cluster.show(truncate=False)

+-------------------------------------------------------+
|cluster                                                |
+-------------------------------------------------------+
|[trudell medicin, aerokat]                             |
|[perfect prep eq, perfect product llc, Perfect-product]|
|[handson, handson equine]                              |
|[apple corp ltd, apple inc]                            |
|[averapack, touch]                                     |
|[aerodawg, trudell]                                    |
|[purr, pro purina]                                     |
+-------------------------------------------------------+



<h2>Brand normalization</h2>

In [29]:

bnt = Brand_normalization(brand_cluster, old_brand_cluster)
new_brand_cluster, updated_brand_cluster = bnt.generate_brand_clusters()
merged_brand_cluster = bnt.get_updated_brand_cluster()

Getting ids for new uuids
Combning dfs
Combning dfs


Combning dfs
Combning dfs


In [165]:
new_brand_cluster.printSchema()
new_brand_cluster.show(truncate=False, n = 50)

root
 |-- id: long (nullable = false)
 |-- brand: string (nullable = false)

+---+---------+
|id |brand    |
+---+---------+
|12 |averapack|
|12 |touch    |
+---+---------+



In [166]:
updated_brand_cluster.printSchema()
updated_brand_cluster.show(truncate=False, n = 50)

root
 |-- id: long (nullable = true)
 |-- brand: string (nullable = false)

+---+-------------------+
|id |brand              |
+---+-------------------+
|1  |trudell medicin    |
|1  |trudell            |
|3  |apple corp ltd     |
|2  |perfect product llc|
|4  |purina             |
|4  |purr               |
+---+-------------------+



In [167]:
merged_brand_cluster.printSchema()
merged_brand_cluster.show(truncate=False, n = 50)

root
 |-- id: long (nullable = true)
 |-- brand: string (nullable = true)

+---+-------------------+
|id |brand              |
+---+-------------------+
|2  |Perfect-product    |
|1  |aerodawg           |
|1  |aerokat            |
|3  |apple              |
|3  |apple inc          |
|7  |dry shod           |
|7  |dryshod west       |
|6  |handson            |
|6  |handson equine     |
|6  |handson equine. Llc|
|2  |perfect prep eq    |
|4  |pro plan           |
|4  |pro purina         |
|1  |trudell medicin    |
|1  |trudell            |
|3  |apple corp ltd     |
|2  |perfect product llc|
|4  |purina             |
|4  |purr               |
|12 |averapack          |
|12 |touch              |
+---+-------------------+



<h1>CONVERT TO DB SCHEMA</h1>

In [40]:
insertion_brnads_a, updation_brnads_a = convert_to_database_schema(new_brand_cluster, brand_dict, db_brand_list, 'BC')
insertion_brnads_b, updation_brnads_b = convert_to_database_schema(updated_brand_cluster, brand_dict,db_brand_list, 'BC')

In [41]:
insertion_brnads_a.show()
updation_brnads_a.show()
insertion_brnads_b.show()
updation_brnads_b.show()


+----------+------------+------+-----------+------------+--------------------+--------------------+-------------+----------+
|cluster_id|crawled_name|source|     status|updated_date|       product_url_1|         image_url_1|product_url_2|query_type|
+----------+------------+------+-----------+------------+--------------------+--------------------+-------------+----------+
|        12|       touch|    BC|IN_PROGRESS|  2022-12-29|https://www.walma...|https://i5.walmar...|         null|         I|
|        12|   averapack|    BC|IN_PROGRESS|  2022-12-29|https://www.walma...|https://i5.walmar...|         null|         I|
+----------+------------+------+-----------+------------+--------------------+--------------------+-------------+----------+

+----------+------------+------+------+------------+-------------+-----------+-------------+----------+
|cluster_id|crawled_name|source|status|updated_date|product_url_1|image_url_1|product_url_2|query_type|
+----------+------------+------+------+--

<h1>SAVING DF TO S3</h1>

In [ ]:
saving_s3_bucket = '/home/preacher/Bungee/CodeRepo/kaushik_forked_repo/kaushik_dev/bnt/data_set/'

In [169]:

new_ids_brand_cluster.show()
# new_ids_brand_cluster = new_ids_brand_cluster.drop('store_list')

+----------+------------+------+-----------+------------+--------------------+--------------------+-------------+
|cluster_id|crawled_name|source|     status|updated_date|       product_url_1|         image_url_1|product_url_2|
+----------+------------+------+-----------+------------+--------------------+--------------------+-------------+
|        12|       touch|    EM|IN_PROGRESS|  2022-12-29|https://www.walma...|https://i5.walmar...|         null|
|        12|   averapack|    EM|IN_PROGRESS|  2022-12-29|https://www.walma...|https://i5.walmar...|         null|
+----------+------------+------+-----------+------------+--------------------+--------------------+-------------+



In [170]:
updated_ids_brand_cluster.show()


+----------+-------------------+------+-----------+------------+--------------------+--------------------+-------------+
|cluster_id|       crawled_name|source|     status|updated_date|       product_url_1|         image_url_1|product_url_2|
+----------+-------------------+------+-----------+------------+--------------------+--------------------+-------------+
|         1|    trudell medicin|    EM|IN_PROGRESS|  2022-12-29|https://www.walma...|https://i5.walmar...|         null|
|         1|            trudell|    EM|IN_PROGRESS|  2022-12-29|https://www.walma...|https://i5.walmar...|         null|
|         3|     apple corp ltd|    EM|IN_PROGRESS|  2022-12-29|https://www.walma...|https://i5.walmar...|         null|
|         2|perfect product llc|    EM|IN_PROGRESS|  2022-12-29|https://www.walma...|https://i5.walmar...|         null|
|         4|             purina|    EM|IN_PROGRESS|  2022-12-29|https://www.walma...|https://i5.walmar...|         null|
|         4|               purr|

In [ ]:
new_ids_brand_cluster_file_key = 'insertion_brand_match_cluster'
saving_df_to_s3(new_ids_brand_cluster, saving_s3_bucket, new_ids_brand_cluster_file_key)

In [ ]:
updated_ids_brand_cluster = 'updation_brand_match_cluster'
saving_df_to_s3(new_ids_brand_cluster)

<h1>STRING CLUSTERING</h1>

In [ ]:
final_brand_cluster.show()

+---+-------------------+
| id|              brand|
+---+-------------------+
|  1|           aerodawg|
|  1|            aerokat|
|  2|    perfect prep eq|
|  2|    Perfect-product|
|  3|              apple|
|  3|          apple inc|
|  4|           pro plan|
|  4|         pro purina|
|  5|             purina|
|  5|               purr|
|  6|handson equine. Llc|
|  6|            handson|
|  6|     handson equine|
|  7|           dry shod|
|  7|       dryshod west|
+---+-------------------+



In [ ]:
str_cluster = String_Clustering(final_brand_cluster, brand_dict)
string_cluster = str_cluster.cluster_unnormalized_brands()
unclustered_brands = str_cluster.get_unclustered_brands()

root
 |-- brand_cluster: array (nullable = true)
 |    |-- element: string (containsNull = true)

Getting ids for new uuids


In [ ]:
string_cluster.show()

+---+--------------------+
| id|               brand|
+---+--------------------+
|  8|Pure Sole Hoof Pr...|
|  8| Pure Sole Products |
+---+--------------------+



In [ ]:
unclustered_brands.show()

+---------+
|    brand|
+---------+
|Goodwinol|
+---------+



<h1> Saving string cluster to s3</h1>

In [ ]:
string_cluster_db_format = convert_to_database_schema(string_cluster, brand_dict, 'SC')
# unclustered_brands_db_format = convert_to_database_schema(unclustered_brands, brand_dict, 'UC')

In [ ]:

string_cluster_db_format.show()

+----------+--------------------+------+-----------+------------+--------------------+--------------------+-------------+
|cluster_id|        crawled_name|source|     status|updated_date|       product_url_1|         image_url_1|product_url_2|
+----------+--------------------+------+-----------+------------+--------------------+--------------------+-------------+
|         8|Pure Sole Hoof Pr...|    SC|IN_PROGRESS|  2022-12-28|https://www.walma...|https://i5.walmar...|         null|
|         8| Pure Sole Products |    SC|IN_PROGRESS|  2022-12-28|https://www.walma...|https://i5.walmar...|         null|
+----------+--------------------+------+-----------+------------+--------------------+--------------------+-------------+



In [ ]:
string_cluster_file_key = 'updation_string_cluster_file_key'
saving_df_to_s3(string_cluster_db_format, saving_s3_bucket, string_cluster_file_key)

In [ ]:
acx.printSchema()

root
 |-- brand: string (nullable = true)



In [ ]:
strcluster.printSchema()

root
 |-- id: long (nullable = false)
 |-- brand: string (nullable = true)



In [ ]:
def get_data_from_aurora_db():
    db_url = "jdbc:postgresql://sandbox-db-cluster.cluster-cf6pgrepnkpy.us-east-1.rds.amazonaws.com:5432/BungeeMatchPool"
    table_name = "brand_standardization"
    db_username = "postgres"
    db_password = "Bungee_sand_rds_2021!"
    df = glueContext.read.format("jdbc").option("url", db_url).option("dbtable", table_name).option("user", db_username).option("password", db_password).load()
    return df

In [ ]:
df = final_brand_cluster.groupBy('cluster_id').agg(collect_set('brand').alias('clusters'))

In [ ]:
df.show()

+----------+--------------------+
|cluster_id|            clusters|
+----------+--------------------+
|         1|[Magnetic Pedigre...|
|         2|[Ark Naturals, Fu...|
|         3|        [3M, 3M,3M™]|
|         4|[4BF For Best Fri...|
|         5|[4x4 North Americ...|
|         6|[A&E Cage Co., A&...|
|         7|   [AA Aquarium, AA]|
|         8|[ABDragons, Black...|
|         9|[CareFRESH, Caref...|
|        10|[ADM Animal Nutri...|
|        11|[AMIGO, Rambo, Ho...|
|        12|[API, Aquarium Ph...|
|        13|[DP Hunter, ASL S...|
|        14|[Worldwide, Absorbz]|
|        15|[Lannett Company,...|
|        16|[Minocycline, Suc...|
|        17|[FELIWAY, Synovi,...|
|        18|[Addiction Foods,...|
|        19|[Adventure Medica...|
|        20|[AERODAWG, AeroDa...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
df = df.filter(size(col("clusters"))<20)


In [ ]:
df.count()

668

In [ ]:
df = df.withColumn("clusters_str",concat_ws(",",col("clusters"))).drop('clusters')


In [ ]:
df.show()

+----------+--------------------+
|cluster_id|        clusters_str|
+----------+--------------------+
|         1|Magnetic Pedigree...|
|         3|           3M,3M,3M™|
|         4|4BF For Best Frie...|
|         5|4x4 North America...|
|         6|A&E Cage Co.,A&E ...|
|         7|      AA Aquarium,AA|
|         8|ABDragons,Black C...|
|         9|CareFRESH,Carefre...|
|        10|ADM Animal Nutrit...|
|        11|AMIGO,Rambo,Horse...|
|        12|API,Aquarium Phar...|
|        13|DP Hunter,ASL Sol...|
|        14|   Worldwide,Absorbz|
|        15|Lannett Company,A...|
|        16|Minocycline,Sucra...|
|        17|FELIWAY,Synovi,Do...|
|        18|Addiction Foods,A...|
|        19|Adventure Medical...|
|        20|AERODAWG,AeroDawg...|
|        21|Alen,Alen Breathe...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
final_brand_cluster.write.mode("overwrite").option("header",True).csv('/home/preacher/Bungee/bnt/data_set/saas_team/cluster_for_back_tracking')

In [ ]:
final_brand_cluster.write.mode("overwrite").option("header",True).parquet('/home/preacher/Bungee/bnt/data_set/saas_team/clusters')

In [ ]:
final_brand_cluster.write.mode("overwrite").option("header",True).csv('/home/preacher/Bungee/bnt/data_set/saas_team/save_to_db')

AnalysisException: CSV data source does not support array<string> data type.

In [ ]:
final_brand_cluster.show()

+----------+--------------------+--------------------+
|cluster_id|               brand|   source_store_list|
+----------+--------------------+--------------------+
|        15|       Acetazolamide|       [chewy_chewy]|
|         2|        Ethical Pets|   [walmart_walmart]|
|       328|            Necoichi|[chewy_chewy, wal...|
|         2|              Zuke's|[chewy_chewy, pet...|
|       251|           long rich|     [amazon_amazon]|
|        27|               Andis|[chewy_chewy, pet...|
|        74|    Bluebonnet Feeds|[chewy_chewy, wal...|
|       161|EQyss Grooming Pr...|       [chewy_chewy]|
|         2|          Fussie Cat|[chewy_chewy, ama...|
|         2|Old Mother Hubbar...|     [target_target]|
|         2|             Prozyme|[entirelypets_ent...|
|         2|                ROAM|[petsmart_petsmar...|
|       410|             Rep-Cal|[chewy_chewy, pet...|
|        89|       Byer of Maine|       [chewy_chewy]|
|         2|    Central Aquatics|   [walmart_walmart]|
|         

In [ ]:
final_brand_cluster.printSchema()
brands = final_brand_cluster.select("cluster_id", 'brand')
brand_set = brands.groupBy('cluster_id').agg(collect_set('brand').alias("brand_list"))

root
 |-- cluster_id: long (nullable = false)
 |-- brand: string (nullable = true)
 |-- store_name_list: array (nullable = false)
 |    |-- element: string (containsNull = false)



In [ ]:
brand_set = brand_set.withColumn('set_length', size(col("brand_list")))

In [ ]:
brand_cluster.printSchema()
brand_set = brand_cluster.withColumn('brand_list', concat_ws(",", col('id_set'))).drop('id_set')
brand_set.show()

root
 |-- id_set: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- cluster_size: integer (nullable = false)



+------------+--------------------+
|cluster_size|          brand_list|
+------------+--------------------+
|        5304|OLA,Vanderpump Pe...|
|           3|Pure Sole Product...|
|           2|     Stroodies,Skoon|
|           2|Contender,KineticVet|
|           2|   Schlueter,Waikato|
|           5|AeroDawg,AeroKat,...|
|           2|Alen BreatheSmart...|
|           2|Goodwinol,Goodwin...|
|           3|HandsOn Equine,Ha...|
|           2|RELIQ,Amerisource...|
|           2|Farrier's Fix,Far...|
|           2|Squire Laboratori...|
|           2|I-Lid 'n Lash,I-M...|
|           2|Van Beek Natural ...|
|           2|FENCEMASTER Kenne...|
|           2|The Petz Kitchen,...|
|           2|This and That Can...|
|           2|    Sassy,Sassy Woof|
|           2|Claudia's Canine ...|
|           2|GLC Direct,GLC Di...|
+------------+--------------------+
only showing top 20 rows



In [ ]:
brand_set.printSchema()

root
 |-- cluster_size: integer (nullable = false)
 |-- brand_list: string (nullable = false)



In [ ]:
brand_set.write.mode("overwrite").option("header",True).csv('/home/preacher/Bungee/bnt/data_set/saas_team/intitial_output')

In [ ]:
brand_matches.printSchema()

root
 |-- brand_a: string (nullable = true)
 |-- brand_b: string (nullable = true)
 |-- count: long (nullable = false)



In [ ]:
print(brand_matches.count())
print(brand_matches.dropna().count())

40909


36942


In [ ]:
brand_matches.write.mode("overwrite").option("header",True).csv('/home/preacher/Bungee/bnt/data_set/saas_team/weighted_brand_matches')

In [ ]:
brand_dict.printSchema()

root
 |-- brand: string (nullable = true)
 |-- brand_info: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- store_name: string (nullable = true)
 |    |    |-- product_url: string (nullable = true)
 |    |    |-- image_url: string (nullable = true)



In [ ]:
brand_dist_to_store = brand_dict.withColumn('brand_info_str', concat_ws(",", col('brand_info'))).drop('brand_info')


AnalysisException: cannot resolve 'concat_ws(',', brand_info)' due to data type mismatch: argument 2 requires (array<string> or string) type, however, 'brand_info' is of array<struct<store_name:string,product_url:string,image_url:string>> type.;
'Project [brand#81, brand_info#17010, concat_ws(,, brand_info#17010) AS brand_info_str#23774]
+- Aggregate [brand#81], [brand#81, collect_set(struct(store_name, store_name#80, product_url, product_url#84, image_url, image_url#83), 0, 0) AS brand_info#17010]
   +- Deduplicate [brand#81, store_name#80]
      +- Project [store_name#80, brand#81, image_url#83, product_url#84]
         +- Filter atleastnnonnulls(1, brand#81)
            +- Deduplicate [sku_uuid#16521]
               +- Project [uuid#79, store_name#80, brand#81, sku#82, image_url#83, product_url#84, create_sku_uuid(uuid#79) AS sku_uuid#16521]
                  +- Filter store_name#80 IN (petcarerx,petsmart,lovegroomers,ryanspet,tractorsupplyrx,californiapetpharmacy,petsuppliesplus,pe

In [ ]:
brand_dict.write.mode("overwrite").option("header",True).parquet('/home/preacher/Bungee/bnt/data_set/saas_team/brand_dict')